In [1]:
%%html
<style>
#explain{
    color:white;background-color:black;padding:10px;
    margin: 10px 0px 10px 0px;
}
#yellow{color:#4AFD32}
</style>

# CHAPTER 6
# Decision Trees

---
# Feature
<pre id="explain">
데이타 개수가 적어도 모델의 신뢰도가 크게 떨어지지 않는다
정규화 과정에 상관 없이 결과가 같다
데이타에 대한 가정이 거의 없다
오버피팅되기 쉽다
파라미터의 개수가 모델을 세우기 전까지는 정해지지 않는다
    <span id="yellow">Nonparametric Model</span>
    <=> <span id="yellow">Parametric Model</span>
        파라미터의 개수가 모델 세우기 전에 정해지고 자유도에 제약이 생긴다
</pre>

---
# Training and Visualizing a Decision Tree

<pre id="explain">
iris 데이타셋에 DecisionTreeClassifier를 적용
</pre>

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
iris = load_iris()
X = iris.data[:, 2:]
y = iris.target
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

<pre id="explain">
DecisionTree를 시각화
output: iris_tree.dot
=> dot파일을 png파일로 변환
</pre>

In [3]:
from sklearn.tree import export_graphviz
export_graphviz(
    tree_clf,
    out_file="iris_tree.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)
import os
os.system("dot -Tpng iris_tree.dot -o iris_tree.png")
os.system("rm iris_tree.dot")

0

---
# Making Predictions
<br>
<img src="1.png" width=600 style="display:inline-block">
<div>
<img src="iris_tree.png" width="300" style="display:inline-block">
<div id="explain" style="display:inline-block;border-radius:30px;">
<p>purity</p>
<p><span id="yellow">$P_{i,k}$</span> : i번째 노드에서 k클래스의 비율</p>
<p>gini:</p>
<p><span id="yellow">$G_i = 1 - \sum_{k=1}^{n}P_{i,k}^2$</span></p>
</div>
</div>

<pre id="explain">
1.  루트노드에서 시작 (depth 0)
2.  루트노드에서 petal length가 2.45cm 보다 작은지 확인 
    <span id="yellow">=> setosa</span>
3.  작으면 왼쪽자식노드, 크면 오른쪽 자식노드 (depth 1)
4.  오른쪽 자식노드에서 petal width가 1.75cm보다 작은지 확인 
    <span id="yellow">=> versicolor</span>
5.  Otherwise 
    <span id="yellow">=> virginica</span>
</pre>

<pre id="explain">
의사결정나무는 분류과정이 직관적으로 이해 가능
<span id="yellow">=> white box model</span>
신경망이나 랜덤포레스트 같은 경우는 이해하기 힘들다
<span id="yellow">=> black box model</span>
</pre>

---
# Estimation Class Probabilties

In [4]:
print("각각 클래스에 대한 확률:\n\t%s"%tree_clf.predict_proba([[5, 1.5]]))
print("가장 높은 확률의 클래스:\n\t%s"%tree_clf.predict([[5, 1.5]]))

각각 클래스에 대한 확률:
	[[ 0.          0.90740741  0.09259259]]
가장 높은 확률의 클래스:
	[1]


<pre id="explain">
<span id="yellow">각각 클래스에 대한 확률 계산</span>
length 5, width 1.5일 경우
    setosa일 확률 0
    versicolor일 확률 49/54 = 0.907
    virginica일 확률 5/54 = 0.093
</pre>

---
# The CART Training Algorithm
### "the Classification And Regression Tree"
<pre id="explain">
<span id="yellow">꼭 2개의 자식만 있어야 하나?</span>
 - 써킷은 <span id="yellow">CART</span> 알고리즘을 쓰는데 이게 Binary트리를 지원한다.
 - 꼭 2개일 필요는 없다. ID3같은 알고리즘을 쓰면 2개 이상의 자식노드도 쓸수 있다.
</pre>
<div id="explain">
<p><span id="yellow">$G_{left/right}$</span>: impurity of left/right subset</p>
<p><span id="yellow">$m_{left/right}$</span>: number of instances in the left/right subset</p>
<p><span id="yellow">$J(k,t_k) = \frac{m_{left}}{m}G_{left}+\frac{m_{right}}{m}G_{right}$</span>: Cost Function</p>
</div>

<pre id="explain">
CART는 그리디 알고리즘이다

그리디알고리즘
    부분을 최적화 한후 합쳐가면서 적당한 최적점을 찾는다
    괜찮은 솔루션을 제시하지만 그게 최적이라고 보장하지는 않는다

왜 최적해를 찾지 않고 그리디알고리즘을 쓰는가?
    최적의 의사결정나무를 찾는 것은 NP-Complete 문제라고 알려져 있기 때문
    시간복잡도가 지수 형태로 연산시간이 너무 오래걸린다
    
시간복잡도
    예측에는 O(log(m))시간 밖에 걸리지 않는다
    모델 생성에는 O(nmlog(m))시간이 걸린다
</pre>

---
# Gini Impurity or Entropy?

<p id="explain">
서킷에서 파라미터로 설정 가능<br><br>
<span id="yellow">$G_i = 1 - \sum_{k=1}^{n}P_{i,k}^2$</span><br><br>
<span id="yellow">$p_{i,k}\neq0$</span><br>
<span id="yellow">$H_i = -\sum_{k=1}^{n}p_{i,k}log(p_{i,k})$</span>
</p>
<pre id="explain">
Gini impurity: 가장 빈도높은 클래스에 치중되는 경향
Entropy: 균형잡힌 의사결정나무를 만드는 편
</pre>

---
# Regularization Hyperparameters

<pre id="explain">
오버피팅을 방지하기 위해 Regularization을 한다
의사결정 나무에서의 Regularization
    나무의 깊이를 제한하기
    분류되는 클래스 안의 인스턴스 수의 최소값을 정하기
    가장 아래 자식 노드의 개수를 제한하기
Pruning
    일단 완벽하게 나눈 다음에 카이제곱검정같은 검정을 통해 가지치기 한다
    보통 5%를 유의수준으로 둔다
</pre>

<img src="2.png" width=600 style="display:inline-block">

---
# Regression
### 의사결정 나무로 회귀분석 하기

In [5]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)
export_graphviz(
    tree_reg,
    out_file="reg_tree.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)
import os
os.system("dot -Tpng reg_tree.dot -o reg_tree.png")
os.system("rm reg_tree.dot")

0

<img src="reg_tree.png" width=300 style="display:inline-block">
<img src="3.png" width=300 style="display:inline-block">

<pre id="explain">
class 대신 value를 예측
gini 대신 mse
</pre>

<img src="4.png" width=600 style="display:inline-block">

### cost function

<img src="5.png" width=600 style="display:inline-block">

### regularization

<img src="6.png" width=600 style="display:inline-block">

---
# Instability

<pre id="explain">
<span id="yellow">Issue 1</span>
의사결정나무는 Orthogonal 한 구분선에 유리하다
따라서 사전에 PCA처리를 하기도 한다
<img src="7.png" width=600 style="display:inline-block">

<span id="yellow">Issue 2</span>
의사결정나무는 training data set의 variation에 매우 민감하다
파란색 class에서 가장 width가 큰 data 한개를 지우니까 모델이 바뀌었다
<img src="8.png" width=600 style="display:inline-block">

<span id="yellow">Issue 3</span>
알고리즘이 node마다 Stochastic한 성질이 있다
=> 같은 training data에 대해서 다른 모델이 생겨날 수 있다
</pre>

